In [ ]:
import diffusers
from diffusers import UNet2DConditionModel
from PIL import Image
import sys
from transformers import CLIPVisionModel, CLIPImageProcessor

sys.path.append("/home/aihao/workspace/StableDiffusionReferenceOnly/src")
from stable_diffusion_reference_only.pipelines.stable_diffusion_reference_only_pipeline import (
    StableDiffusionReferenceOnlyPipeline,
)
from stable_diffusion_reference_only.models.unet_2d_dobule_condition import (
    UNet2DDobuleConditionModel,
)
from diffusers.schedulers import DDPMScheduler
import json
from torchsummary import summary

In [ ]:
with open(
    "/home/aihao/workspace/StableDiffusionReferenceOnly/src/stable_diffusion_reference_only/models/unet.json"
) as f:
    unet_config = json.load(f)

In [ ]:
unet_config

In [ ]:
unet=UNet2DDobuleConditionModel.from_config(unet_config)

In [ ]:
sdxl_unet = UNet2DConditionModel.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", subfolder="unet"
)

In [ ]:
unet_parameters = unet.state_dict()
sdxl_unet_parameters = sdxl_unet.state_dict()

In [ ]:
for key in unet_parameters:
    if key in sdxl_unet_parameters:
        if unet_parameters[key].shape == sdxl_unet_parameters[key].shape:
            unet_parameters[key] = sdxl_unet_parameters[key]
        else:
            unet_parameters[key] = sdxl_unet_parameters[key][
                0 : unet_parameters[key].shape[0],
                0 : unet_parameters[key].shape[1],
            ]

In [ ]:
unet.load_state_dict(unet_parameters)

In [ ]:
# vae = diffusers.AutoencoderKL.from_pretrained("stabilityai/sdxl-vae")
vae = diffusers.AutoencoderKL.from_pretrained(
    "stabilityai/stable-diffusion-2-1", subfolder="vae"
)

In [ ]:
image_encoder = CLIPVisionModel.from_pretrained(
    "laion/CLIP-ViT-bigG-14-laion2B-39B-b160k"
)

In [ ]:
clip_image_processor = CLIPImageProcessor.from_pretrained(
    "laion/CLIP-ViT-bigG-14-laion2B-39B-b160k"
)

In [ ]:
scheduler = DDPMScheduler.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", subfolder="scheduler"
)

In [ ]:
pipe = StableDiffusionReferenceOnlyPipeline(
    vae, image_encoder, clip_image_processor, unet, scheduler
)

In [ ]:
pipe.save_pretrained(
    "/home/aihao/workspace/DeepLearningContent/models/sd_reference_only/init"
)